<a href="https://colab.research.google.com/github/mpily/basic-ml-course/blob/Assignment3/Solution3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import numpy as np

In [13]:
from sklearn.datasets import load_iris
data = load_iris()
X, Y = data.data, data.target

In [14]:
def choose_two_classes(X,Y):
  mask_idx = Y < 2
  X        = X[mask_idx]
  Y        = Y[mask_idx]
  return X,Y

In [15]:
X, Y = choose_two_classes(X,Y)
print(X.shape, Y.shape)

(100, 4) (100,)


$$\mu = \frac{1}{N} \sum_{i = 1} ^{N} x_i$$
$$\sigma = \sqrt{\frac{1}{N} \sum_{i = 1}^{N} (x_i - \mu)^{2}}$$

In [16]:
import math
def get_means(array):
  elements_sum = sum(array)
  return elements_sum/len(array)

def std_dev(array, mean):
  var  = 0
  for element in array:
    var += ((element - mean) ** 2)
  var /= len(array)
  std = math.sqrt(var)
  return std

def get_z_value(x,means,std_devs):
  z = np.array([(x[i] - means[i])/std_devs[i] for i in range(len(x))])
  return z

In [17]:
#testing formulas
#mean after applying formula for z value should be 0
temp = [5 ,3, 6, 1, 4]
means = np.array([get_means(temp)])
print(means)
devs = np.array([std_dev(temp,means)])
print(devs)
temp_zs = [get_z_value([i],means,devs) for i in temp]
print(temp_zs)
new_mean = get_means(temp_zs)
print(new_mean)


[3.8]
[1.72046505]
[array([0.69748583]), array([-0.46499055]), array([1.27872403]), array([-1.62746694]), array([0.11624764])]
[1.33226763e-16]


In [18]:
x_means  = [get_means(X[:,i]) for i in range(len(X[0]))]
x_stdevs = [std_dev(X[:,i],x_means[i]) for i in range(len(X[0]))]
print(X.shape)
for i in range(len(X)):
  X[i] = get_z_value(X[i],x_means,x_stdevs)
print(X.shape)

(100, 4)
(100, 4)


In [20]:
#testing to make sure that all means after applying z conversion are 0
x_means  = [get_means(X[:,i]) for i in range(len(X[0]))]
print(x_means)

[-4.385380947269368e-15, -4.485301019485632e-16, -1.1879386363489176e-16, 3.363975764614224e-16]


In [21]:
appended_column = np.array([[1] for i in range(X.shape[0])])
X               = np.hstack((appended_column,X))
assert X.shape == (100,5)
#print(X)

In [24]:
weight  = np.array([0 for i in range(X.shape[1])])
print (weight.shape)
print(X.shape)
print(weight)

assert weight.shape == (5,)
#ensuring all multiplications result in correct output shape as expected
print((X @ weight).shape)

(5,)
(100, 5)
[0 0 0 0 0]
(100,)


sigmoid function:
$$\sigma(z) = \frac{1}{1 + e^{-z}}$$

In [25]:
def sigmoid(z):
  sigmoid = 1/(1 + np.exp(-z))
  return sigmoid

In [26]:
#testing sigmoid function as well as loss function
x_temp = np.array([0.5,0.3,0.4,0.6])
x_pret = np.array([0.2,0.3,0.4,0.5])
print(sigmoid(x_temp))
print(-np.sum(x_temp * np.log(x_pret) + (1 - x_temp) * np.log(1 - x_pret)) / x_temp.size)
print(0.5 * np.log(0.2) + (1 - 0.5) * np.log(1 - 0.2))
print(0.3 * np.log(0.3) + (1 - 0.3) * np.log(1 - 0.3))
print(0.4 * np.log(0.4) + (1 - 0.4) * np.log(1 - 0.4))
print(0.6 * np.log(0.5) + (1 - 0.6) * np.log(1 - 0.5))
print(- 0.916290731874155 -0.6108643020548935 -0.6730116670092565-0.6931471805599453)
print(-2.8933138814982504/4)

[0.62245933 0.57444252 0.59868766 0.64565631]
0.7233284703745626
-0.916290731874155
-0.6108643020548935
-0.6730116670092565
-0.6931471805599453
-2.8933138814982504
-0.7233284703745626


In [27]:
def accuracy(X, Y, weight):
    '''
    Evaluate the model, represented by `weight`, with data (X, Y).
    
    Input:
        X:      data features
        Y:      data labels
        weight: model weights
    Ouput:
        Model accuracy on input data.
    '''
    # implement your code here. Below are only some suggestion. 
    # You don't necessary have to follow the exact step

    #Step 1: get the model prediction

    #Step 2: Threshold the probability value to get the class. 

    #Step 3: Compare your model predicted classes with the label y

    #step 4: calculate the accuracy using the formula above
    Y_prediction = X @ weight
    print(Y_prediction.shape)
    Y_prediction = np.array([int(sigmoid(Y_prediction[i]) > 0.5) for i in range(len(Y_prediction))])
    print(Y_prediction, Y)
    correct_predictions = 0
    for i in range(len(Y)):
      if Y_prediction[i] == Y[i]:
        correct_predictions += 1
    print(correct_predictions)
    return correct_predictions/len(Y)


Steps include:


1.  Calculate the model $$h_{\theta}(X) = \sigma(X\theta)$$
2.  Calculate the loss/cost function (In this case, the Binary Cross Entropy Function is chosen) $$J(\theta) = \frac{-1}{m} \sum_{i = 1}^{m} y_{i} (ln h_{\theta}(x_{i}) + (1 - y_{i}) ln (1 - h_{\theta}(x_{i})))$$
3.  Calculate the gradient:
$$\frac{\partial J(\theta)}{\partial \theta} = \frac{1}{m} X^{T} ( Y - h_{\theta}(X))$$
derivation:
Let the terms under the sum be denoted by $F(\theta,i)$
$$F(\theta,i) = y_{i}ln(h_{\theta}(x_{i})) + ln(1 - h_{\theta}(x_{i})) - y_{i}ln(1 - h_{\theta} (x_{i}))$$
$$F(\theta,i) = y_{i}ln( \frac{h_{\theta}(x_{i})}{1 - h_{\theta}(x_{i})}) + ln(1 - h_{\theta}(x_{i}))$$
$$=y_{i}(ln \frac{\frac{1}{1 + e^{-\theta X_{i}}}}{1 - \frac{1}{1 + e^{-\theta X_{i}}}}) + ln \frac{e^{-\theta X_{i}}}{1 + e^{-\theta X_{i}}}$$
$$= y_{i} (ln(e^{-\theta X_{i}})) + ln \frac{e^{-\theta X_{i}}}{1 + e^{-\theta X_{i}}}$$
Thus:
$$\frac{\partial J(\theta)}{\partial \theta} = \frac{-1}{m}\sum_{i = 1}^{m} \frac{\partial F}{\partial \theta} $$
$$\frac{\partial F}{\partial \theta} = y_{i} x_{i} + x_{i}h_{\theta}$$

4.  Update the model parameter
  $$\theta = \theta - \alpha \frac{\partial J}{\partial \theta}$$

5.  Calcualte the model accuracy (use the function you implemented above)



In [28]:
#Hyper parameters:
#Don't change these
# learning rate
alpha = 1e-2
# epochs
epoch = 10000

trick by Nguyen Hung Quang to calculate loss to with numerical stability
rewrite equation as:
 $$-y_{i} \cdot log(1 + e^{-\theta X}) - \theta \cdot X_{i}+ (1 - y_{i})\cdot log(1 + e^{-\theta X}) $$
 the only log we need to compute is $log(1 + e ^{-\theta X})$  which can be easily calculated using numpy.log1p

In [52]:
#
#
def train_model(y_target, x,theta):
  for ep in range(epoch):
    y_predict = x @ theta
    loss      = np.sum(y_target * np.log1p(np.exp(-y_predict)) + (1 - y_target) * (np.log1p(np.exp(-y_predict)) + y_predict)) / y_target.size
    print(loss)
    y_predict = sigmoid(y_predict)
    gradient  =  X.T @ (y_predict - y_target)/y_target.size
    theta = theta - alpha  * gradient
  return theta


In [32]:
#solution using library implementation
#100% accuracy
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X,Y)
model.coef_ , model.intercept_
model.score(X,Y)

1.0

In [55]:
#loss consistently decreases to a minimum value of 0.00589
weight  = np.array([0 for i in range(X.shape[1])])
print (weight)
assert weight.shape == (5,)
weight = train_model(Y,X,weight)
print(weight)

Streaming output truncated to the last 5000 lines.
0.011006455925675807
0.011004463589212622
0.011002472016529808
0.011000481207181294
0.010998491160721538
0.010996501876705313
0.010994513354687685
0.010992525594223996
0.010990538594870241
0.010988552356182391
0.010986566877717057
0.010984582159030946
0.010982598199681304
0.010980614999225601
0.010978632557221826
0.010976650873228156
0.010974669946803104
0.010972689777505562
0.010970710364894904
0.010968731708530532
0.0109667538079726
0.010964776662781298
0.010962800272517172
0.010960824636741289
0.010958849755015036
0.010956875626899952
0.010954902251958039
0.01095292962975171
0.010950957759843581
0.010948986641796641
0.01094701627517437
0.010945046659540372
0.010943077794458762
0.010941109679493879
0.01093914231421044
0.010937175698173514
0.01093520983094854
0.010933244712101213
0.010931280341197598
0.010929316717804165
0.010927353841487625
0.010925391711815051
0.010923430328353936
0.010921469690671983
0.010919509798337258
0.01091755

In [56]:
print(accuracy(X,Y,weight))

(100,)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1] [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
100
1.0


In [57]:
#checking expected values result.
weight_target = np.array([0.12047504, -0.44156746, -0.89309501, 2.965364, 3.3427994])
print(accuracy(X,Y,weight_target))

(100,)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1] [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
100
1.0
